In [1]:
!pip install requests

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [7]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 63.3 MB/s eta 0:00:00


In [8]:
%%writefile pdftotext.py
import pdfplumber

with pdfplumber.open('your_file.pdf') as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += page.extract_text()

Writing pdftotext.py


In [4]:
%%writefile app.py
import streamlit as st
import requests
import json

import streamlit as st
import pandas as pd
import requests
import json

import pdfplumber

with pdfplumber.open('your_file.pdf') as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += page.extract_text()

# Function to call the Azure OpenAI API
def chat_with_gpt(endpoint, api_key, prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    data = {
        "model": "gpt-4o",  # Specify the model
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 800,  # Adjust token limit as needed
    }

    response = requests.post(f"{endpoint}", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code}, {response.text}"

# Streamlit user interface
st.title("ChatGPT-4o Azure Interface with File Upload")

# Input fields for endpoint and API key
endpoint = st.text_input("Azure OpenAI Endpoint", placeholder="https://<your-resource-name>.openai.azure.com/")
api_key = st.text_input("API Key", type="password")

# File uploader for multiple file types
uploaded_file = st.file_uploader("Upload a file (txt, csv)", type=["txt", "csv"])

# Text input for user prompt
user_prompt = st.text_area("Ask a question related to the uploaded file:")

# Button to submit the prompt
if st.button("Submit"):
    if endpoint and api_key and uploaded_file and user_prompt:
        # Read the uploaded file
        if uploaded_file.type == "text/plain":
            content = uploaded_file.read().decode("utf-8")
            file_type = "text"
        elif uploaded_file.type == "text/csv":
            content = pd.read_csv(uploaded_file).to_string()  # Convert DataFrame to string
            file_type = "csv"
        else:
            st.warning("Currently, only text and CSV files are processed.")
            content = ""

        if content:
            # Combine the content and prompt
            full_prompt = f"Here is the content of the {file_type} file:\n{content}\n\nUser's Question: {user_prompt}"
            response = chat_with_gpt(endpoint, api_key, full_prompt)
            st.write("**Response:**")
            st.write(response)
    else:
        st.warning("Please fill all fields and upload a file.")

Overwriting app.py


In [20]:
%%writefile pdfapp.py
import streamlit as st
import pdfplumber
import openai

# Set up Azure OpenAI API credentials
endpoint = st.text_input("Azure OpenAI Endpoint", placeholder="https://<your-resource-name>.openai.azure.com/")
api_key = st.text_input("API Key", type="password")

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to call the Azure OpenAI API
def chat_with_gpt(endpoint, api_key, prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    data = {
        "model": "gpt-4o",  # Specify the model
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 800,  # Adjust token limit as needed
    }

    response = requests.post(f"{endpoint}", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code}, {response.text}"

# Streamlit app UI
st.title("PDF to GPT-4 Prompt and Response")
st.write("Upload a PDF and get a response from GPT-4 based on the content.")

# File uploader widget
pdf_file = st.file_uploader("Upload your PDF", type=["pdf"])

if pdf_file:
    # Extract text from the uploaded PDF
    with st.spinner("Extracting text from PDF..."):
        extracted_text = extract_text_from_pdf(pdf_file)

    if extracted_text:
        #st.subheader("Extracted Text from PDF")
        #st.write(extracted_text[:1000] + "...")  # Show the first 1000 characters

        # Create a prompt from the extracted text
        prompt = st.text_area("Enter a prompt for GPT-4 based on the PDF content:")  # Default prompt from first 500 chars
        full_prompt = f"Here is the content of the file:\n{extracted_text}\n\nUser's Question: {prompt}"
        if st.button("Submit to GPT-4"):
            if full_prompt:
                with st.spinner("Generating response..."):
                    # Query GPT-4 with the prompt
                    response = chat_with_gpt(full_prompt)
                st.subheader("GPT-4 Response:")
                st.write(response)
            else:
                st.warning("Please provide a prompt.")
    else:
        st.error("Could not extract text from the PDF.")

Overwriting pdfapp.py


In [15]:
!wget -q -O - ipv4.icanhazip.com

34.106.180.208


In [ ]:
!streamlit run pdfapp.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.180.208:8501

your url is: https://upset-ties-rule.loca.lt


In [18]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.180.208:8501

your url is: https://busy-parents-cover.loca.lt
  Stopping...
^C
